## Getting your data!

In [1]:
import os
import fun
import pandas as pd
import warnings
import numpy as np
from function import kmean
warnings.filterwarnings('ignore')

dir = '../data/csv/'
files = os.listdir(dir)
if (len(files) < 3) :
    fun.downloadcsv()
df1 = pd.read_csv(dir+files[0])
df2 = pd.read_csv(dir+files[1])
df3 = pd.read_csv(dir+files[2])


In [2]:
dataset = pd.merge(df1, df2, on='track_id', how='inner')
dataset = pd.merge(dataset, df3, on='track_id', how='inner')
print(dataset.shape, df1.shape, df2.shape, df3.shape)

(13129, 820) (13129, 250) (106574, 519) (106574, 53)


## Q2.2

In [3]:
dataset_Num = dataset[dataset.T[dataset.dtypes!=np.object].index]
dataset_Numf = dataset_Num.dropna()
dataset_Num.drop_duplicates()
dataset_Num.shape

(13129, 780)

In [4]:
cc = kmean(dataset_Num[dataset_Num.columns.values])
d = cc.fit(3)
print(cc.point)
print(d)

[[5.16999772e-02 5.89315221e-01 8.79136147e-01 ... 8.54438687e-01
  3.05548956e-01 6.20910864e-01]
 [4.02357297e-01 9.17164851e-01 8.50233630e-01 ... 1.46472916e-01
  1.01771359e-01 1.36313512e-01]
 [2.37789293e-04 1.44851538e-01 8.51256912e-01 ... 3.19350739e-01
  6.91493187e-01 4.60967284e-01]]
[0 0 0 ... 0 0 0]


In [5]:
print(2 in d)

False
